In [1]:
# Styling notebook
from IPython.core.display import display, HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

C:\Users\Renata\AppData\Local\Temp\ipykernel_8396\1617930524.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Inicio

## Links uteis 

- Cores em graficos plotly: https://plotly.com/python/discrete-color/
- https://www.selecolor.com/en/recommended-color-palette/

## Importação de libs

In [2]:
# para ler arquivo excel (xls)
# conda install -c anaconda xlrd

import datetime
from datetime import datetime
import os

# Analise
import pandas as pd 
import numpy as np 

#Visualization
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns 
import plotly
import plotly.express as px

## Configurações

In [3]:
# para habilitar rolagem horizontal do dataframe 
# caso não funcione no Firefox
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

In [4]:
# formatação de mensagens de print
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
#print(color.BOLD + 'Hello World !' + color.END)

## Parâmetros

In [5]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 200
pd.set_option('max_colwidth', 1000) # tamanho das colunas para exibição do conteúdo de textos longos

np.set_printoptions(threshold=1000)

# Formatar numeros float (2 casas decimais)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

# Formatações para gráficos
sns.set(style="darkgrid") #cor da area dos plots
figsize = (10,10)

# Definição de funções

## checar_df

Para um determinado dataframe, são verificados:
- total de registros;
- quantidade e tipos de colunas;
- quantidade de valores únicos;
- quantidade de valores nulos (para identificar a necessidade de pre processamento dos dados).

In [6]:
def checar_df(df):
    print(color.BOLD  + color.RED + 'Informações do dataframe: ' + color.END)
    df.info()
    
    print(color.BOLD + color.RED + "\nNúmero de observacoes na base de dados: " + color.END, df.shape[0])
    print('Total de ', df.shape[1], 'Colunas: ', df.columns.values.tolist())
    
    print(color.BOLD + color.RED + "\nNúmero de valores unicos (Não inclui NaN): " + color.END)
    display(df.nunique(dropna=True).to_frame(name='Contagem Unicos').sort_index())
    
    print(color.BOLD + color.RED + "Número de valores nulos:" + color.END)
    display(df.isnull().sum().sort_values(ascending=False).to_frame(name='Contagem Nulos').sort_index())

## define_tp_dimensao

In [7]:
def define_tp_dimensao(x):
    # para simplificar: considerar presencial/ à distância
    if x == 1:   return 'Cursos presenciais'
    elif x == 2: return 'Cursos à distância'
    elif x == 3: return 'Cursos à distância'
    elif x == 4: return 'Cursos à distância'
    else: return 'não informado'

## define_tp_cat_adm

In [8]:
def define_tp_cat_adm(x):
    if x == 1:   return 'Pública Federal'
    elif x == 2: return 'Pública Estadual'
    elif x == 3: return 'Pública Municipal'
    elif x == 4: return 'Privada com fins lucrativos'
    elif x == 5: return 'Privada sem fins lucrativos'
    elif x == 6: return 'Privada - Particular em sentido estrito'
    elif x == 7: return 'Especial'        
    elif x == 8: return 'Privada comunitária'        
    elif x == 9: return 'Privada confessional'        
    else: return 'não informado'

## define_tp_org_acad

In [9]:
def define_tp_org_acad(x):
    if x == 1: return 'Universidade'    
    elif x == 2: return 'Centro Universitário'    
    elif x == 3: return 'Faculdade'
    elif x == 4: return 'Instituto Federal de Educação, Ciência e Tecnologia'
    elif x == 5: return 'Centro Federal de Educação Tecnológica'        
    else: return 'não informado'

## define_tp_org_principal

In [10]:
def define_tp_org_principal(x):
    if x == 1: return 'Faculdade'               #'Universidade'    
    elif x == 2: return 'Centro Universitário'  #'Centro Universitário'    
    elif x == 3: return 'Faculdade'             #'Faculdade'
    elif x == 4: return 'Outros'  # 'Instituto Federal de Educação, Ciência e Tecnologia' 
    elif x == 5: return  'Outros' #'Centro Federal de Educação Tecnológica'        
    else: return 'não informado'

## define_tp_rede

In [11]:
def define_tp_rede(x):
    if x == 1: return 'Pública'  
    elif x == 2: return 'Privada'
    else: return 'não informado'

## define_tp_grau_academico

In [12]:
def define_tp_grau_academico(x):
    if x == 1: return 'Bacharelado'           
    elif x == 2: return 'Licenciatura'  
    elif x == 3: return 'Tecnológico'   
    elif x == 4: return 'Bacharelado e Licenciatura' 
    else: return 'não aplicavel'

## define_tp_mod_ensino

In [13]:
def define_tp_mod_ensino(x):
    if x == 1: return 'Presencial'           
    elif x == 2: return 'A distância'  
    else: return 'não aplicavel'

## define_tp_nivel_acad

In [14]:
def define_tp_nivel_acad(x):
    if x == 1: return 'Graduacao'           
    elif x == 2: return 'Sequencial Formacao Especifica'  
    else: return 'não aplicavel'

# Carga dos Dados 

## Carga dados IES

In [15]:
ies = pd.read_csv('./dados/preparados/dados_ies_consolidado.csv', sep='|', 
                  low_memory=False)

In [16]:
# selecionar todas colunas que começam com CO_ e outras (transformar em string) 
colunas_CO = [col for col in ies if col.startswith('CO_')] + ['COD_IBGE', 'IN_CAPITAL_IES','NU_CEP_IES']
dict_dtype = {column : 'str'  for column in colunas_CO}

ies = pd.read_csv('./dados/preparados/dados_ies_consolidado.csv', sep='|', 
                  dtype = dict_dtype, 
                  low_memory=False)

ies.head(3).T

,0,1,2
NU_ANO_CENSO,2022,2022,2022
NO_REGIAO_IES,Centro-Oeste,Centro-Oeste,Centro-Oeste
CO_REGIAO_IES,5,5,5
NO_UF_IES,Mato Grosso,Mato Grosso,Mato Grosso
SG_UF_IES,MT,MT,MT
CO_UF_IES,51,51,51
NO_MUNICIPIO_IES,Cuiabá,Cuiabá,Cuiabá
IN_CAPITAL_IES,1,1,1
NO_MESORREGIAO_IES,Centro-Sul Mato-grossense,Centro-Sul Mato-grossense,Centro-Sul Mato-grossense
CO_MESORREGIAO_IES,4,4,4


In [17]:
checar_df(ies)

Informações do dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595 entries, 0 to 2594
Data columns (total 94 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   NU_ANO_CENSO                       2595 non-null   int64  
 1   NO_REGIAO_IES                      2595 non-null   object 
 2   CO_REGIAO_IES                      2595 non-null   object 
 3   NO_UF_IES                          2595 non-null   object 
 4   SG_UF_IES                          2595 non-null   object 
 5   CO_UF_IES                          2595 non-null   object 
 6   NO_MUNICIPIO_IES                   2595 non-null   object 
 7   IN_CAPITAL_IES                     2595 non-null   object 
 8   NO_MESORREGIAO_IES                 2595 non-null   object 
 9   CO_MESORREGIAO_IES                 2595 non-null   object 
 10  NO_MICRORREGIAO_IES                2595 non-null   object 
 11  CO_MICRORREGIAO_IES          

,Contagem Unicos
CAPITAL,2
COD_IBGE,719
CO_IES,2595
CO_MANTENEDORA,1769
CO_MESORREGIAO_IES,15
CO_MICRORREGIAO_IES,66
CO_REGIAO_IES,5
CO_UF_IES,27
DS_COMPLEMENTO_ENDERECO_IES,893
DS_ENDERECO_IES,2217


Número de valores nulos:


,Contagem Nulos
CAPITAL,0
COD_IBGE,0
CO_IES,0
CO_MANTENEDORA,0
CO_MESORREGIAO_IES,0
CO_MICRORREGIAO_IES,0
CO_REGIAO_IES,0
CO_UF_IES,0
DS_COMPLEMENTO_ENDERECO_IES,1
DS_ENDERECO_IES,0


## Carga Dados Cursos

In [18]:
# para executar algum ano especifico
ano = '2022'
filename = f'./dados/MICRODADOS_CADASTRO_CURSOS_{ano}.CSV'
filename

'./dados/MICRODADOS_CADASTRO_CURSOS_2022.CSV'

In [19]:
%%time
cursos_orig = pd.read_csv(filename, 
                          #dtype = dict_dtype,
                          sep=';',
                          encoding='iso-8859-1',
                          low_memory=False)

print('Total registros: ', cursos_orig.shape[0])
print('Total colunas: ', cursos_orig.shape[1])

Total registros:  573019
Total colunas:  200
CPU times: total: 11.4 s
Wall time: 11.5 s


In [20]:
# selecionar todas colunas que começam com CO_ e outras (transformar em string) 
# facilita operações de merge (tem que conter os mesmos tipos de dados)
colunas_CO = [col for col in cursos_orig if col.startswith('CO_')] + ['IN_CAPITAL','IN_GRATUITO']
dict_dtype = {column : 'str'  for column in colunas_CO}

cursos_orig = pd.read_csv(filename, 
                          dtype = dict_dtype,
                          sep=';',
                          encoding='iso-8859-1',
                          low_memory=False)
print('Total registros: ', cursos_orig.shape[0]) #573019
print('Total colunas: ', cursos_orig.shape[1]) #200

Total registros:  573019
Total colunas:  200


In [21]:
#cursos_orig.info(max_cols=500)

In [22]:
#checar_df(cursos_orig)

In [23]:
#cursos_orig.sample(5)

In [24]:
#col_interesse = []
#cursos = cursos_orig[col_interesse].copy()
#cursos = cursos_orig.copy()

## Carga dados Indicadores Municipais (Atlas) - 2010

In [25]:
# definir os tipos de algumas colunas
list_col_string = ['Codmun6','Codmun7'] 
dict_dtype = {column : 'str'  for column in list_col_string}

df_ind_atlas = pd.read_csv('./dados/selecao_indicadores_sociais_municipios.csv', 
                       sep='|', dtype = dict_dtype, low_memory=False) 

# considerar apenas a ultima referencia (2010)
df_ind_atlas = df_ind_atlas[df_ind_atlas['ANO']==2010]

col_excluir = ['ANO']
for c in col_excluir: del df_ind_atlas[c]

# renomear colunas 
df_ind_atlas = df_ind_atlas.rename(columns = {
                                    'Codmun6':'CodIBGE_6',
                                    'Codmun7':'CodIBGE',
                                    'pesotot':'Pop_total',
                                    'pesourb':'Pop_urbana',
                                    'POP':'Pop_total_dom_part',    
                                    'POPT':'Pop_total_dom_part_exc_renda_nula'    
                                })

In [26]:
col_relevantes = ['CodIBGE', 'Pop_total','Pop_urbana','Pop_total_dom_part', 'Pop_total_dom_part_exc_renda_nula']
df_ind_atlas[col_relevantes].head(5).T

,11130,11131,11132,11133,11134
CodIBGE,1100015,1100023,1100031,1100049,1100056
Pop_total,24392,90353,6313,78574,17029
Pop_urbana,13970,76525,2693,61921,14419
Pop_total_dom_part,24097,89766,6303,78141,16942
Pop_total_dom_part_exc_renda_nula,22429,88730,6156,76876,16815


In [27]:
df_ind_atlas.shape

(5565, 111)

In [28]:
# checar as diferentes infos de Pop de um municipio
# display(ibge[ibge['NOME_MUNICIPIO']=='Brasília'])

# col = ['Município','Pop_total','Pop_urbana','Pop_total_dom_part','Pop_total_dom_part_exc_renda_nula']
# df_ind_atlas[df_ind_atlas['CodIBGE']=='5300108'][col]

# Tratamento dados 

In [29]:
#col_interesse = []
#cursos = cursos_orig[col_interesse].copy()
cursos = cursos_orig.copy()

## Enriquecimento de dados

### Adicionar Descrição do Tipo de Dimensão

In [30]:
# Convertendo coluna TP_DIMENSAO
cursos.insert(cursos.columns.get_loc('TP_DIMENSAO')+1, \
                'Tipo_Dimensao', cursos['TP_DIMENSAO'].apply(lambda x: define_tp_dimensao(x)))

### Adicionar Descrição Tipo Organização Acadêmica

In [31]:
# Convertendo coluna TP_ORGANIZACAO_ACADEMICA
cursos.insert(cursos.columns.get_loc('TP_ORGANIZACAO_ACADEMICA')+1, \
                'Tipo_Org_Acad', cursos['TP_ORGANIZACAO_ACADEMICA'].apply(lambda x: define_tp_org_acad(x)))

### Adicionar Organizacao Academica principal: Faculdade, Centro e Outros

In [32]:
# Simplificando o conceito de Organização administrativa
cursos.insert(cursos.columns.get_loc('Tipo_Org_Acad')+1, \
                'Tipo_Org_Principal', cursos['TP_ORGANIZACAO_ACADEMICA'].apply(lambda x: define_tp_org_principal(x)))

### Adicionar Descrição Tipo Categoria Administrativa

In [33]:
# Convertendo coluna TP_CATEGORIA_ADMINISTRATIVA
cursos.insert(cursos.columns.get_loc('TP_CATEGORIA_ADMINISTRATIVA')+1, \
                'Tipo_Cat_Admn', cursos['TP_CATEGORIA_ADMINISTRATIVA'].apply(lambda x: define_tp_cat_adm(x)))

### Adicionar Descrição Tipo Rede

In [34]:
# Convertendo coluna TP_REDE
cursos.insert(cursos.columns.get_loc('TP_REDE')+1, \
                'Tipo_Rede', cursos['TP_REDE'].apply(lambda x: define_tp_rede(x)))

### Adicionar Descrição Tipo Grau Academico

In [35]:
# Convertendo coluna TP_GRAU_ACADEMICO
cursos.insert(cursos.columns.get_loc('TP_GRAU_ACADEMICO')+1, \
                'Tipo_Grau_Acad', cursos['TP_GRAU_ACADEMICO'].apply(lambda x: define_tp_grau_academico(x)))

### Adicionar Descrição Tipo Modalidade de Ensino

In [36]:
# Convertendo coluna TP_MODALIDADE_ENSINO
cursos.insert(cursos.columns.get_loc('TP_MODALIDADE_ENSINO')+1, \
                'Tipo_Mod_Ens', cursos['TP_MODALIDADE_ENSINO'].apply(lambda x: define_tp_mod_ensino(x)))

### Adicionar Descrição Nivel Academico

In [37]:
# Convertendo coluna TP_NIVEL_ACADEMICO
cursos.insert(cursos.columns.get_loc('TP_NIVEL_ACADEMICO')+1, \
                'Tipo_Niv_Acad', cursos['TP_NIVEL_ACADEMICO'].apply(lambda x: define_tp_nivel_acad(x)))

## Checar quantidades e percentuais de cada categoria

In [38]:
print('Total de registros - cursos:', cursos.shape[0])

Total de registros - cursos: 573019


In [39]:
# checar tipos dos dados
# confirmar que sejam todos int
col = ['TP_DIMENSAO', 'TP_ORGANIZACAO_ACADEMICA', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_REDE',
      'TP_GRAU_ACADEMICO', 'TP_MODALIDADE_ENSINO', 'TP_NIVEL_ACADEMICO', ]
cursos[col].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573019 entries, 0 to 573018
Data columns (total 7 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   TP_DIMENSAO                  573019 non-null  int64
 1   TP_ORGANIZACAO_ACADEMICA     573019 non-null  int64
 2   TP_CATEGORIA_ADMINISTRATIVA  573019 non-null  int64
 3   TP_REDE                      573019 non-null  int64
 4   TP_GRAU_ACADEMICO            573019 non-null  int64
 5   TP_MODALIDADE_ENSINO         573019 non-null  int64
 6   TP_NIVEL_ACADEMICO           573019 non-null  int64
dtypes: int64(7)
memory usage: 30.6 MB


In [40]:
display(cursos[['TP_DIMENSAO','Tipo_Dimensao']].value_counts())
cursos[['TP_DIMENSAO','Tipo_Dimensao']].value_counts(normalize=True)

TP_DIMENSAO  Tipo_Dimensao     
2            Cursos à distância    527328
1            Cursos presenciais     36089
3            Cursos à distância      9207
4            Cursos à distância       395
Name: count, dtype: int64

TP_DIMENSAO  Tipo_Dimensao     
2            Cursos à distância   0.92
1            Cursos presenciais   0.06
3            Cursos à distância   0.02
4            Cursos à distância   0.00
Name: proportion, dtype: float64

In [41]:
display(cursos[['TP_ORGANIZACAO_ACADEMICA','Tipo_Org_Acad']].value_counts())
cursos[['TP_ORGANIZACAO_ACADEMICA','Tipo_Org_Acad']].value_counts(normalize=True)

TP_ORGANIZACAO_ACADEMICA  Tipo_Org_Acad                                      
1                         Universidade                                           279865
2                         Centro Universitário                                   266868
3                         Faculdade                                               23990
4                         Instituto Federal de Educação, Ciência e Tecnologia      2235
5                         Centro Federal de Educação Tecnológica                     61
Name: count, dtype: int64

TP_ORGANIZACAO_ACADEMICA  Tipo_Org_Acad                                      
1                         Universidade                                          0.49
2                         Centro Universitário                                  0.47
3                         Faculdade                                             0.04
4                         Instituto Federal de Educação, Ciência e Tecnologia   0.00
5                         Centro Federal de Educação Tecnológica                0.00
Name: proportion, dtype: float64

In [42]:
display(cursos[['TP_CATEGORIA_ADMINISTRATIVA','Tipo_Cat_Admn']].value_counts())
cursos[['TP_CATEGORIA_ADMINISTRATIVA','Tipo_Cat_Admn']].value_counts(normalize=True)

TP_CATEGORIA_ADMINISTRATIVA  Tipo_Cat_Admn              
4                            Privada com fins lucrativos    493177
5                            Privada sem fins lucrativos     61226
1                            Pública Federal                  9781
2                            Pública Estadual                 7485
3                            Pública Municipal                1001
7                            Especial                          349
Name: count, dtype: int64

TP_CATEGORIA_ADMINISTRATIVA  Tipo_Cat_Admn              
4                            Privada com fins lucrativos   0.86
5                            Privada sem fins lucrativos   0.11
1                            Pública Federal               0.02
2                            Pública Estadual              0.01
3                            Pública Municipal             0.00
7                            Especial                      0.00
Name: proportion, dtype: float64

In [43]:
display(cursos[['TP_REDE','Tipo_Rede']].value_counts())
cursos[['TP_REDE','Tipo_Rede']].value_counts(normalize=True)

TP_REDE  Tipo_Rede
2        Privada      554403
1        Pública       18616
Name: count, dtype: int64

TP_REDE  Tipo_Rede
2        Privada     0.97
1        Pública     0.03
Name: proportion, dtype: float64

In [44]:
display(cursos[['TP_GRAU_ACADEMICO','Tipo_Grau_Acad']].value_counts())
cursos[['TP_GRAU_ACADEMICO','Tipo_Grau_Acad']].value_counts(normalize=True)

TP_GRAU_ACADEMICO  Tipo_Grau_Acad
3                  Tecnológico       270752
1                  Bacharelado       183827
2                  Licenciatura      117033
0                  não aplicavel       1407
Name: count, dtype: int64

TP_GRAU_ACADEMICO  Tipo_Grau_Acad
3                  Tecnológico      0.47
1                  Bacharelado      0.32
2                  Licenciatura     0.20
0                  não aplicavel    0.00
Name: proportion, dtype: float64

In [45]:
display(cursos[['TP_MODALIDADE_ENSINO','Tipo_Mod_Ens']].value_counts())
cursos[['TP_MODALIDADE_ENSINO','Tipo_Mod_Ens']].value_counts(normalize=True)

TP_MODALIDADE_ENSINO  Tipo_Mod_Ens
2                     A distância     536930
1                     Presencial       36089
Name: count, dtype: int64

TP_MODALIDADE_ENSINO  Tipo_Mod_Ens
2                     A distância    0.94
1                     Presencial     0.06
Name: proportion, dtype: float64

In [46]:
display(cursos[['TP_NIVEL_ACADEMICO','Tipo_Niv_Acad']].value_counts())
cursos[['TP_NIVEL_ACADEMICO','Tipo_Niv_Acad']].value_counts(normalize=True)

TP_NIVEL_ACADEMICO  Tipo_Niv_Acad                 
1                   Graduacao                         573010
2                   Sequencial Formacao Especifica         9
Name: count, dtype: int64

TP_NIVEL_ACADEMICO  Tipo_Niv_Acad                 
1                   Graduacao                        1.00
2                   Sequencial Formacao Especifica   0.00
Name: proportion, dtype: float64

## Filtrar registros de interesse (escopo de estudo dos CURSOS)

filtrar para 
- Cursos presenciais	- TP_DIMENSAO = 1
- Org Academica			- TP_ORGANIZACAO_ACADEMICA = todos
- Publ/ Privada			- TP_CATEGORIA_ADMINISTRATIVA in (1,4,5,6,8,9) (somente federais e todas as privadas)
- Publ/ Privada			- TP_REDE = todos
- Cursos de NS 			- TP_GRAU_ACADEMICO in (1,2,3,4)
- Cursos presenciais 	- TP_MODALIDADE_ENSINO = 1
- Somente graduacao		- TP_NIVEL_ACADEMICO = 1

In [47]:
cursos_escopo = cursos[
                (cursos['TP_DIMENSAO']== 1 ) &                                     # apenas cursos presenciais
                (cursos['TP_CATEGORIA_ADMINISTRATIVA'].isin([1,4,5,6,8,9]) ) &     # apenas publicas federais e todas as privadas
                (cursos['TP_GRAU_ACADEMICO'].isin([1,2,3,4]) ) &                   # bacharelado, licenciatura, tecnologo
                (cursos['TP_MODALIDADE_ENSINO']==1) &                              # somente presenciais
                (cursos['TP_NIVEL_ACADEMICO']==1)]                                 # somente graduacao

In [48]:
print('Total de registros - cursos:', cursos.shape[0])
print('Total de registros - cursos_escopo:', cursos_escopo.shape[0])

Total de registros - cursos: 573019
Total de registros - cursos_escopo: 31871


## Retirada de registros com todas informações zeradas

In [49]:
# retirada de cursos com vagas = 0 ? (nao retirar pois tem matriculados)
col = ['QT_INSCRITO_TOTAL','QT_ING','QT_MAT']
cursos_escopo[cursos_escopo['QT_VG_TOTAL']==0][col]# .shape # 1953

,QT_INSCRITO_TOTAL,QT_ING,QT_MAT
9622,0,0,93
9703,0,0,207
9842,0,0,1
9935,0,0,0
9942,0,0,0
...,...,...,...
571208,0,0,2
571228,0,0,2
571255,0,0,0
571842,0,0,5


In [50]:
# checar casos de todas colunas de matriculas, vagas, etc. zeradas
col = ['QT_INSCRITO_TOTAL','QT_ING','QT_MAT','QT_CONC']
cursos_escopo[
            (cursos_escopo['QT_VG_TOTAL']==0) &
            (cursos_escopo['QT_INSCRITO_TOTAL']==0) &
            (cursos_escopo['QT_ING']==0) &
            (cursos_escopo['QT_MAT']==0)][col]#.shape # 562

,QT_INSCRITO_TOTAL,QT_ING,QT_MAT,QT_CONC
9935,0,0,0,0
9942,0,0,0,0
9948,0,0,0,0
10064,0,0,0,0
10095,0,0,0,0
10109,0,0,0,0
10114,0,0,0,0
10168,0,0,0,0
10199,0,0,0,0
10200,0,0,0,0


In [51]:
# Verificar as informações
idx = cursos_escopo[
            (cursos_escopo['QT_VG_TOTAL']==0) &
            (cursos_escopo['QT_INSCRITO_TOTAL']==0) &
            (cursos_escopo['QT_ING']==0) &
            (cursos_escopo['QT_MAT']==0)][col].index

len(idx)

562

In [52]:
cursos_escopo.loc[idx].head(10).T

,9935,9942,9948,10064,10095,10109,10114,10168,10199,10200
NU_ANO_CENSO,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
NO_REGIAO,Centro-Oeste,Centro-Oeste,Centro-Oeste,Centro-Oeste,Centro-Oeste,Centro-Oeste,Centro-Oeste,Centro-Oeste,Centro-Oeste,Centro-Oeste
CO_REGIAO,5,5,5,5,5,5,5,5,5,5
NO_UF,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal
SG_UF,DF,DF,DF,DF,DF,DF,DF,DF,DF,DF
CO_UF,53,53,53,53,53,53,53,53,53,53
NO_MUNICIPIO,Brasília,Brasília,Brasília,Brasília,Brasília,Brasília,Brasília,Brasília,Brasília,Brasília
CO_MUNICIPIO,5300108,5300108,5300108,5300108,5300108,5300108,5300108,5300108,5300108,5300108
IN_CAPITAL,1,1,1,1,1,1,1,1,1,1
TP_DIMENSAO,1,1,1,1,1,1,1,1,1,1


In [53]:
#ies.loc[idx]['NO_IES'].value_counts()

In [54]:
#ies[ies['CO_IES']=='1113']['NO_IES']

In [55]:
# Retirada destes registros
cursos_escopo = cursos_escopo.drop(idx).reset_index(drop=True)

print('Total de registros - cursos:', cursos.shape[0])
print('Total de registros - cursos_escopo:', cursos_escopo.shape[0])

Total de registros - cursos: 573019
Total de registros - cursos_escopo: 31309


## Salvar dados Cursos consolidados em CSV

In [56]:
# salvar em arquivo
# cursos.to_csv('./dados/preparados/dados_cursos_consolidado.csv', sep='|', index=False)
# cursos_escopo.to_csv('./dados/preparados/dados_cursos_escopo_consolidado.csv', sep='|', index=False)

In [57]:
file_csv = f'./dados/preparados/dados_cursos_escopo_consolidado_{ano}.csv'
file_csv

'./dados/preparados/dados_cursos_escopo_consolidado_2022.csv'

In [58]:
#cursos_escopo.to_csv(file_csv, sep='|', index=False)